In [1]:
from twelvedata import TDClient
import pandas as pd
import time
from sqlalchemy import inspect
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
td = TDClient(apikey="faa93033c87f4b8290b9486920e97a86")

In [2]:
# Cryptocurrencies List, td.get_cryptocurrencies_list() see at: (https://twelvedata.com/blog/get-high-quality-financial-data-directly-into-python)
all_Cryptos = td.get_cryptocurrencies_list().as_json()

#Print firtst item in list as visual aid
all_Cryptos[0]

{'symbol': '0xBTC/BTC',
 'available_exchanges': ['Hotbit', 'Mercatox'],
 'currency_base': '0xBitcoin',
 'currency_quote': 'Bitcoin'}

In [3]:
# Filter the symbols that are being compared to USD
usd_symbols = [crypto['symbol'] for crypto in all_Cryptos if '/USD' in crypto['symbol']]
print("Number of USD-paired symbols:", len(usd_symbols))

Number of USD-paired symbols: 1641


In [4]:
# Time Series example
ts = td.time_series(
    symbol="BTC/USD",
    timezone="America/New_York",
    interval="30min",
    outputsize=5
).as_pandas()
ts

,open,high,low,close
datetime,,,,
2023-07-27 21:00:00,29266.47,29278.84,29260.47,29270.67
2023-07-27 20:30:00,29234.46,29300.00,29223.55,29266.48
2023-07-27 20:00:00,29214.93,29251.58,29203.10,29234.47
2023-07-27 19:30:00,29193.06,29221.35,29193.06,29214.92
2023-07-27 19:00:00,29193.83,29194.20,29171.35,29193.06


In [5]:
# Create a list to store the DataFrames for each symbol in the batch
crypto_list = []

# Loop through batches of symbols
symbol_batch = ['BTC/USD', 'ETH/USD', 'USDT/USD', 'XRP/USD', 
                'BNB/USD', 'ADA/USD', 'SOL/USD', 'DOGE/USD']

# Process each symbol in the current batch
for symbol in symbol_batch:
    try:
        ts = td.time_series(
            symbol=symbol,
            interval="1week",
            timezone="America/New_York",
            outputsize=208
        ).with_bbands(time_period=3).with_macd(fast_period=4, slow_period=8, signal_period=2).with_rsi(time_period=6).as_pandas()

        # Add a new numeric index column without dropping the original timestamp index
        ts.reset_index(inplace=True)

        # Add the DataFrame for the current symbol to the list
        crypto_list.append(ts)

    except Exception as e:
        print(f"Error fetching data for symbol '{symbol}': {str(e)}")

# Introduce a delay of 1 minute between API calls
time.sleep(60)

print("Total DataFrames fetched:", len(crypto_list))

Total DataFrames fetched: 8


In [6]:
crypto_list[1]

,datetime,open,high,low,close,upper_band,middle_band,lower_band,macd,macd_signal,macd_hist,rsi
0,2023-07-24,1888.81000,1890.94000,1833.52000,1861.73000,1940.59729,1890.97333,1841.34938,7.39940,10.51998,-3.12058,50.09670
1,2023-07-17,1921.95000,1935.97000,1846.84000,1888.80000,1940.08518,1891.35333,1842.62149,15.90139,16.76113,-0.85974,54.00389
2,2023-07-10,1862.64000,2029.66000,1846.52000,1922.39000,1972.32294,1907.69333,1843.06373,20.77873,18.48060,2.29813,58.74129
3,2023-07-03,1937.83000,1977.16000,1827.00000,1862.87000,1960.98186,1899.76333,1838.54480,14.47613,13.88433,0.59179,52.60154
4,2023-06-26,1898.98000,1959.99000,1815.84000,1937.82000,2041.19584,1852.44333,1663.69083,20.88747,12.70075,8.18673,62.33571
...,...,...,...,...,...,...,...,...,...,...,...,...
93,2021-10-11,3413.80005,3966.47998,3377.53003,3847.54004,3966.50171,3560.35335,3154.20500,0.00000,0.00000,0.00000,0.00000
94,2021-10-04,3416.73999,3667.88989,3274.86011,3415.52002,3631.98430,3299.05330,2966.12231,0.00000,0.00000,0.00000,0.00000
95,2021-09-27,3058.71997,3484.60010,2783.83008,3418.00000,3571.20103,3270.18327,2969.16550,0.00000,0.00000,0.00000,0.00000
96,2021-09-20,3328.98999,3346.88989,2653.58008,3063.63989,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000


In [7]:
# Create a database connection using SQLAlchemy
engine = create_engine('sqlite:///crypto_data.db')

Base = automap_base()
Base.prepare(autoload_with=engine)

# Loop through the DataFrames in 'crypto_list' and the symbols in 'symbol_batch'
for df, symbol in zip(crypto_list, symbol_batch):
    table_name = f'{symbol.replace("/", "_")}'  # Create a unique table name based on the symbol
    df.to_sql(name=table_name, con=engine, if_exists='replace', index=False)
# Close the database connection
#engine.dispose()

In [3]:
# Create a database connection using SQLAlchemy
engine = create_engine('sqlite:///crypto_data.db')

# Get the table names from the database using the inspect() function
inspector = inspect(engine)
tables = inspector.get_table_names()

# Print the table names
print(tables)

['ADA_USD', 'BNB_USD', 'BTC_USD', 'DOGE_USD', 'ETH_USD', 'SOL_USD', 'USDT_USD', 'XRP_USD']
